In [ ]:
import * as tslab from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf-8");
tslab.display.html(`<style>${css}</style>`);

# A parser for Propositional Logic

In [ ]:
const lexSpec = /([ \t]+)|([A-Za-z][A-Za-z0-9<>,]*)|([⊤⊥∧∨¬→↔⊕()])/g;

The function `tokenize(s)` transform the string `s` into an array of tokens.  The string s
is supposed to represent a formula from propositional logic.

In [ ]:
function tokenize(s: string): string[] {
    return Array.from(s.matchAll(lexSpec))
        .map(([_, ws, identifier, operator]) => identifier || operator)
        .filter((token): token is string => !!token);
}

The function `isPropVar(s)` checks, whether the string `s` can be interpreted as a propositional variable.

In [ ]:
function isPropVar(s: string): boolean {
    return /^[A-Za-z][A-Za-z0-9<>,]*$/.test(s);
}

The `popOrThrow(stack, errorMsg)` helper ensures type safety by immediately throwing an error if `pop()` returns `undefined`, thereby guaranteeing a non-nullable return value without relying on unsafe type assertions.

In [ ]:
function popOrThrow<T>(stack: T[], errorMsg: string): T {
    const val = stack.pop();
    if (val === undefined) {
        throw new Error(errorMsg);
    }
    return val;
}

In [ ]:
type Variable = string
type Formula  = Variable | ['⊤' | '⊥'] | ['¬', Formula] | ['↔' | '→' | '∧' | '∨', Formula, Formula];

The class `LogicParser` implements the shunting yard algorithm to parse formulas from
propositional logic.  The strings that represent formulas are transformed
into nested tuples that are interpreted as syntax trees representing the 
formulas.

In [ ]:
class LogicParser {
    private _tokens:    string[];
    private _operators: string[];
    private _arguments: Formula[];
    private _input:    string;

    constructor(s: string) {
        this._tokens = tokenize(s).reverse();
        this._operators = [];
        this._arguments = [];
        this._input = s;
    }

    parse(): Formula {
        while (this._tokens.length !== 0) {
            const next_op = popOrThrow(this._tokens, "Unexpected end of input"); 
            if (isPropVar(next_op)) {
                this._arguments.push(next_op);
                continue;
            }
            if (next_op === '⊤' || next_op === '⊥') {
                this._operators.push(next_op);
                continue;
            }
            if (this._operators.length === 0 || next_op === '(') {
                this._operators.push(next_op);
                continue;
            }
            const stack_op = this._operators[this._operators.length - 1];
            if (stack_op === '(' && next_op === ')') {
                this._operators.pop();
            } else if (next_op === ')' || this._eval_before(stack_op, next_op)) {
                this._pop_and_evaluate();
                this._tokens.push(next_op);
            } else {
                this._operators.push(next_op);
            }
        }
        while (this._operators.length !== 0) {
            this._pop_and_evaluate();
        }
        if (this._arguments.length !== 1) {
            throw new Error(`could not parse ${this._input}`);
        }
        return popOrThrow(this._arguments, "Unexpected end of input");
    }

    private _eval_before(stack_op: string, next_op: string): boolean {
        // Check if the operator on top of the operator stack should be evaluated
        // before the next operator from the input array.
        if (stack_op === '(') return false;
        const precedences: { [key: string]: number } = {
            '↔': 1, '→': 2, '∨': 4, '∧': 5, '¬': 6, '⊤': 7, '⊥': 7
        };
        if (precedences[stack_op] > precedences[next_op]) {
            return true;
        } else if (precedences[stack_op] === precedences[next_op]) {
            if (stack_op === next_op) {
                return stack_op === '∧' || stack_op === '∨';
            }
            return true;
        }
        return false;
    }

    private _pop_and_evaluate(): void {
        const op = popOrThrow(this._operators, "Unexpected end of input");
        if (op === '⊤' || op === '⊥') {
            this._arguments.push([op]);
            return;
        }
        if (op === '¬') {
            const arg = this._arguments.pop()!;
            this._arguments.push(['¬', arg]);
            return;
        }
        if (op == '↔' || op == '→' || op == '∧' || op == '∨') {
                const rhs = popOrThrow(this._arguments, "Unexpected end of input");
                const lhs = popOrThrow(this._arguments, "Unexpected end of input");
                this._arguments.push([op, lhs, rhs]);
        }
    }

    toString(): string {
        // Return the current state as a string for pretty printing.
        return `${this._tokens.toString()} ${this._arguments.toString()} ${this._operators.toString()}`;
    }
}

In [ ]:
function testParser(s: string): void {
    const p = new LogicParser(s);
    console.log('\n');
    console.log('parsing', s);
    console.log(p.parse());
}
function runTests() {
    testParser('¬⊥');
    testParser('¬p ↔ (p → ⊥)');
    testParser('¬⊥ ↔ ⊤');
    testParser('p ∧ q');
    testParser('p ∨ q ∧ r');
    testParser('p ∧ q ∨ r');
    testParser('p ∧ q → r ∨ s');
    testParser('p → q → r');
    testParser('p ∧ q ↔ q ∨ p');
    testParser('¬(p ∨ q) ↔ ¬p ∨ ¬q');
    testParser('a<1,2> ↔ b<2,1>');
}
runTests();

In [ ]:
testParser('p ↔ q ↔ r')

In [ ]:
testParser("a ∧ b ∧ c")